# Drawing Parallel with Master's Thesis - WebRTC Prototype

- WebRTC specification
- Essence of a strong coupling
- Essence of distributed nature of the application
- Is p2p a viable option for data pipelines